# 1. Silver Layer: Create totals and followers views

In [0]:
%sql
CREATE VIEW IF NOT EXISTS IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_TOTALS_TABLE) AS 
    SELECT date, impressions, engagements FROM IDENTIFIER (:BRONZE_CATALOG || "." || :BRONZE_SCHEMA || "." || :BRONZE_TOTALS_TABLE);
CREATE VIEW IF NOT EXISTS IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_FOLLOWERS_TABLE) AS 
    SELECT date, new_followers FROM IDENTIFIER (:BRONZE_CATALOG || "." || :BRONZE_SCHEMA || "." || :BRONZE_FOLLOWERS_TABLE);